# 🚀 KLOOK 통합 크롤러 시스템

## 📋 지원하는 모든 시나리오:
1. **기본 크롤링**: 도시 선택 → 탭별 순차 크롤링 (전체→투어→티켓→교통→기타)
2. **랭킹 기반 수집**: 각 탭별 1~50위 순서대로 수집, 중복 자동 건너뛰기
3. **세션 연속성**: 중간 중단 후 이어서 계속 가능
4. **hashlib 고속 중복 체크**: 이미 크롤링한 URL 초고속 검사
5. **페이지네이션**: KLOOK 15개/페이지 구조 완벽 대응
6. **CSV 자동 저장**: 10개마다 자동 저장

---

## 🔧 1. 시스템 초기화 및 설정

In [ ]:
# 필수 라이브러리 import 및 시스템 체크
import os
import sys
import time
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# 모듈 import
try:
    from klook_modules.control_system import KlookMasterController
    from klook_modules.system_utils import setup_driver, check_dependencies, get_system_info
    from klook_modules.config import UNIFIED_CITY_INFO, CONFIG
    from klook_modules.url_manager import is_url_already_processed, mark_url_as_processed
    from klook_modules.tab_selector import execute_integrated_tab_selector_system
    from klook_modules.url_collection import collect_urls_with_pagination
    from klook_modules.crawler_engine import KlookCrawlerEngine
    from klook_modules.ranking_manager import ranking_manager  # 랭킹 매니저 추가
    print("✅ 모든 모듈 로드 성공!")
except ImportError as e:
    print(f"❌ 모듈 로드 실패: {e}")
    print("klook_modules 폴더가 현재 디렉토리에 있는지 확인하세요.")

# 시스템 상태 체크
print("\n🔍 시스템 상태 체크...")
dependencies = check_dependencies()
missing_deps = [lib for lib, available in dependencies.items() if not available]

if missing_deps:
    print(f"❌ 누락된 라이브러리: {', '.join(missing_deps)}")
    print("다음 명령으로 설치하세요: pip install selenium chromedriver-autoinstaller pandas requests beautifulsoup4")
else:
    print("✅ 모든 의존성 준비 완료!")

print(f"\n🌍 지원 도시: {len(UNIFIED_CITY_INFO)}개")
print(f"📊 설정 상태: hashlib={CONFIG.get('USE_HASH_SYSTEM', True)}, V2_URL={CONFIG.get('USE_V2_URL_SYSTEM', True)}")
print(f"🏆 랭킹 매니저: {'✅ 로드됨' if 'ranking_manager' in locals() else '❌ 로드 실패'}")
print(f"💾 32개 컬럼 구조: ✅ 적용됨")
print(f"📸 듀얼 이미지 시스템: ✅ 적용됨")

## 🎛️ 2. 크롤링 설정 위젯

In [ ]:
# 크롤링 설정 위젯 생성
print("🎛️ 크롤링 설정을 선택하세요:")

# 도시 입력 (직접 입력 가능)
city_widget = widgets.Text(
    value='서울',
    placeholder='도시명을 입력하세요 (예: 서울, 도쿄, 방콕)',
    description='도시:',
    style={'description_width': 'initial'}
)

# 순위 범위 설정
start_rank_widget = widgets.IntText(
    value=1,
    description='시작 순위:',
    min=1,
    style={'description_width': 'initial'}
)

end_rank_widget = widgets.IntText(
    value=50,
    description='종료 순위:',
    min=1,
    style={'description_width': 'initial'}
)

# 크롤링 모드 선택 (테스트 모드 제거)
mode_widget = widgets.RadioButtons(
    options=[
        ('🎯 전체 크롤링 (모든 탭)', 'full'),
        ('⚡ 전체 탭만', 'only_all'),
        ('🔄 이어서 계속', 'resume')
    ],
    value='full',
    description='크롤링 모드:',
    style={'description_width': 'initial'}
)

# 모드별 설명 출력
print("\n📋 크롤링 모드 설명:")
print("🎯 전체 크롤링: 전체→투어&액티비티→티켓&입장권→교통→기타 순서로 모든 탭 크롤링")
print("⚡ 전체 탭만: '전체' 탭에서만 크롤링 (빠른 수집용)")
print("🔄 이어서 계속: 이전에 중단된 크롤링을 이어서 계속")

print("\n📊 순위 설정 설명:")
print("시작 순위 1, 종료 순위 50: 각 탭에서 1~50위 상품 수집")
print("시작 순위 51, 종료 순위 100: 각 탭에서 51~100위 상품 수집 (확장 크롤링)")

# 고급 옵션 (필수 옵션들만)
advanced_options = widgets.VBox([
    widgets.Checkbox(value=True, description='10개마다 CSV 자동 저장'),
    widgets.Checkbox(value=True, description='메인+썸네일 이미지 다운로드', disabled=False),
    widgets.Checkbox(value=True, description='세션 상태 저장')
])

print("\n📋 고급 옵션 설명:")
print("💾 10개마다 CSV 자동 저장: 크롤링 중 10개씩 완료될 때마다 자동으로 CSV 파일에 저장")
print("📸 메인+썸네일 이미지 다운로드: 상품의 메인 이미지와 썸네일 이미지를 다운로드 (썸네일 없으면 메인만)")
print("💾 세션 상태 저장: 크롤링 진행 상황을 저장하여 중단 시 이어서 계속할 수 있도록 함")
print("✅ 중복 URL 건너뛰기: 항상 활성화 (이미 크롤링한 URL은 자동으로 건너뜀)")
print("🔄 중복 상품 랭킹 업데이트: 중복 발견 시 랭킹 정보만 누적 업데이트")

# 위젯 표시
settings_box = widgets.VBox([
    city_widget,
    widgets.HBox([start_rank_widget, end_rank_widget]),
    mode_widget,
    widgets.HTML("<b>고급 옵션:</b>"),
    advanced_options
])

display(settings_box)

# 설정 확인 함수
def get_current_settings():
    return {
        'city': city_widget.value.strip(),
        'start_rank': start_rank_widget.value,
        'end_rank': end_rank_widget.value,
        'mode': mode_widget.value,
        'skip_duplicates': True,  # 항상 활성화
        'auto_save': advanced_options.children[0].value,
        'download_images': advanced_options.children[1].value,
        'save_session': advanced_options.children[2].value
    }

print("\n⬆️ 위에서 설정을 선택한 후 다음 셀을 실행하세요.")

## 🚀 3. 크롤링 실행

In [ ]:
# 현재 설정 가져오기
settings = get_current_settings()
print("📋 현재 설정:")
for key, value in settings.items():
    print(f"   {key}: {value}")

# 사용자 확인
confirm = input("\n위 설정으로 크롤링을 시작하시겠습니까? (y/n): ")
if confirm.lower() != 'y':
    print("❌ 크롤링이 취소되었습니다. 위 셀에서 설정을 다시 조정하세요.")
else:
    print("✅ 크롤링을 시작합니다!")
    
    # 드라이버 설정
    print("\n🔧 크롬 드라이버 설정 중...")
    driver = setup_driver()
    
    # 글로벌 변수로 설정 저장 (다음 셀들에서 사용)
    CURRENT_CITY = settings['city']
    START_RANK = settings['start_rank']
    END_RANK = settings['end_rank'] 
    CRAWLING_MODE = settings['mode']
    
    print(f"🎯 '{CURRENT_CITY}' 크롤링 준비 완료!")
    print(f"📊 순위 범위: {START_RANK}위 ~ {END_RANK}위")
    print(f"🎮 모드: {CRAWLING_MODE}")
    print(f"🏆 랭킹 매니저: 중복 URL 자동 처리")
    print(f"💾 32개 컬럼 구조: 자동 적용")
    print(f"📸 듀얼 이미지: 메인+썸네일 자동 다운로드")

## 📋 4. 탭별 랭킹 수집 (1단계: URL 수집)

In [ ]:
# 탭별 랭킹 URL 수집
print(f"🎪 '{CURRENT_CITY}' 탭별 랭킹 URL 수집 시작!")
print("=" * 70)

# KLOOK 탭 순서
TAB_ORDER = ['전체', '투어&액티비티', '티켓&입장권', '교통', '기타']

# 모드별 탭 선택
if CRAWLING_MODE == 'only_all':
    selected_tabs = ['전체']
else:
    selected_tabs = TAB_ORDER

# 순위 범위 설정
RANKING_RANGE = END_RANK - START_RANK + 1

print(f"📊 처리할 탭: {selected_tabs}")
print(f"🎯 순위 범위: {START_RANK}위 ~ {END_RANK}위 ({RANKING_RANGE}개)")
print(f"🏆 랭킹 매니저: 중복 URL 자동 누적")

# 수집된 URL 저장용
collected_urls_by_tab = {}
all_collected_urls = set()  # 중복 체크용

try:
    # 1. KLOOK 메인 페이지 이동
    from klook_modules.driver_manager import go_to_main_page, find_and_fill_search, click_search_button
    
    print("\n🌐 KLOOK 메인 페이지 이동...")
    go_to_main_page(driver)
    
    print(f"🔍 '{CURRENT_CITY}' 검색 중...")
    find_and_fill_search(driver, CURRENT_CITY)
    click_search_button(driver)
    
    time.sleep(3)  # 검색 결과 로딩 대기
    
    # 2. 탭별 처리를 위한 탭 감지
    from klook_modules.tab_selector import detect_klook_tabs, process_tab
    
    print("\n🔍 KLOOK 탭 구조 감지 중...")
    detected_tabs = detect_klook_tabs(driver)
    
    if not detected_tabs:
        print("❌ 탭 감지 실패, 기본 페이지네이션으로 대체")
        # 기존 페이지네이션 방식 사용
        for tab_name in selected_tabs:
            if tab_name == '전체':
                # 순위 범위에 맞는 페이지 계산 (15개/페이지)
                max_pages = (END_RANK + 14) // 15  # 올림 계산
                
                tab_urls = collect_urls_with_pagination(
                    driver, 
                    CURRENT_CITY, 
                    max_pages=max_pages, 
                    strategy="smart"
                )
                
                # 순위 범위에 맞는 URL만 선택
                selected_urls = tab_urls[START_RANK-1:END_RANK]  # 인덱스는 0부터 시작
                collected_urls_by_tab[tab_name] = selected_urls
                all_collected_urls.update(selected_urls)
                
                # 랭킹 매니저에 저장
                try:
                    ranking_manager.save_tab_ranking(selected_urls, CURRENT_CITY, tab_name, "pagination")
                    print(f"    🏆 랭킹 정보 저장 완료: {len(selected_urls)}개")
                except Exception as e:
                    print(f"    ⚠️ 랭킹 저장 실패: {e}")
                
                break
    else:
        print("✅ 탭 구조 감지 성공!")
        
        # 3. 각 탭별 URL 수집 (탭 전환 포함)
        for tab_name in selected_tabs:
            print(f"\n🎪 '{tab_name}' 탭 처리 중...")
            
            if tab_name not in detected_tabs:
                print(f"    ⚠️ '{tab_name}' 탭을 찾을 수 없습니다.")
                collected_urls_by_tab[tab_name] = []
                continue
            
            # 탭 처리 (자동 클릭 + URL 수집 + 랭킹 저장)
            tab_info = detected_tabs[tab_name]
            tab_info["ranking_limit"] = END_RANK  # 종료 순위까지 수집
            tab_info["start_rank"] = START_RANK   # 시작 순위 설정
            tab_info["end_rank"] = END_RANK       # 종료 순위 설정
            
            result = process_tab(driver, tab_name, tab_info, CURRENT_CITY)
            
            if result.get("success", False):
                tab_urls = result.get("urls", [])
                
                # 순위 범위에 맞는 URL 선택
                if len(tab_urls) >= START_RANK:
                    selected_urls = tab_urls[START_RANK-1:END_RANK]  # 순위 범위 슬라이싱
                else:
                    selected_urls = tab_urls  # 수집된 URL이 시작 순위보다 적은 경우 전부 사용
                
                # 중복 체크 (랭킹 매니저 활용)
                unique_tab_urls = []
                for rank, url in enumerate(selected_urls, START_RANK):
                    if settings['skip_duplicates'] and url in all_collected_urls:
                        print(f"    ⏭️ 중복 URL 발견 ({rank}위): 랭킹 정보만 누적")
                        # 중복 URL도 랭킹 정보는 저장해야 함
                        continue
                    unique_tab_urls.append(url)
                    all_collected_urls.add(url)
                
                collected_urls_by_tab[tab_name] = unique_tab_urls
                print(f"  ✅ '{tab_name}' 탭 수집 완료: {len(unique_tab_urls)}개 URL ({START_RANK}~{START_RANK+len(unique_tab_urls)-1}위)")
                print(f"  🏆 랭킹 매니저: 중복 URL {len(selected_urls) - len(unique_tab_urls)}개 누적 저장")
            else:
                print(f"  ❌ '{tab_name}' 탭 처리 실패: {result.get('error', '알 수 없음')}")
                collected_urls_by_tab[tab_name] = []
            
            # 다음 탭으로 이동 전 대기
            if tab_name != selected_tabs[-1]:
                print(f"  🔄 다음 탭으로 이동 중...")
                time.sleep(2)

except Exception as e:
    print(f"❌ URL 수집 중 오류: {e}")
    import traceback
    traceback.print_exc()

# 수집 결과 요약
print("\n🎉 URL 수집 완료!")
print("=" * 70)
total_urls = sum(len(urls) for urls in collected_urls_by_tab.values())
print(f"📊 총 수집 URL: {total_urls}개")
print(f"🎯 순위 범위: {START_RANK}위 ~ {END_RANK}위")

for tab_name, urls in collected_urls_by_tab.items():
    print(f"   🎪 {tab_name}: {len(urls)}개")

# 랭킹 매니저 통계
try:
    ranking_stats = ranking_manager.get_city_ranking_stats(CURRENT_CITY)
    if ranking_stats:
        print(f"\n🏆 랭킹 매니저 통계:")
        print(f"   총 URL: {ranking_stats.get('total_urls', 0)}개")
        print(f"   중복 URL: {ranking_stats.get('duplicate_urls', 0)}개")
        print(f"   처리된 탭: {len(ranking_stats.get('tabs_processed', []))}개")
except Exception as e:
    print(f"⚠️ 랭킹 통계 조회 실패: {e}")

print("\n⬇️ 다음 셀에서 상세 크롤링을 시작하세요.")

## 🔥 5. 상세 크롤링 실행 (2단계: 데이터 수집)

In [ ]:
# 상세 크롤링 시작
print(f"🔥 '{CURRENT_CITY}' 상세 크롤링 시작!")
print("=" * 70)

# 크롤링 엔진 초기화 (driver를 전달하고 통계 초기화)
crawler = KlookCrawlerEngine(driver)
crawler.reset_stats(CURRENT_CITY)

# 전체 진행 상황 추적
total_planned = sum(len(urls) for urls in collected_urls_by_tab.values())
total_completed = 0
total_skipped = 0
total_failed = 0

# 배치 저장을 위한 데이터 수집
batch_data = []
batch_size = 10 if settings['auto_save'] else 50

start_time = time.time()

print(f"💾 32개 컬럼 구조: 자동 적용")
print(f"📸 듀얼 이미지 시스템: {'✅ 활성화' if settings.get('download_images', True) else '❌ 비활성화'}")
print(f"🏆 랭킹 매니저: 중복 URL 스마트 처리")
print(f"💾 세션 저장: {'✅ 활성화' if settings.get('save_session', True) else '❌ 비활성화'}")
print(f"💾 자동 백업: ✅ 20개마다 + 최종 백업")

try:
    # 탭별 순차 크롤링
    for tab_name, urls in collected_urls_by_tab.items():
        if not urls:
            continue
            
        print(f"\n🎪 '{tab_name}' 탭 크롤링 시작 ({len(urls)}개 URL)")
        print("-" * 50)
        
        tab_completed = 0
        tab_skipped = 0
        tab_failed = 0
        
        # URL별 순차 크롤링 (랭킹 순서 보장)
        for rank, url in enumerate(urls, START_RANK):
            current_progress = total_completed + tab_completed + 1
            
            print(f"\n📊 진행률: {current_progress}/{total_planned} | {tab_name} {rank}위")
            print(f"🔗 URL: {url[:60]}...")
            
            # 랭킹 매니저 중복 체크
            try:
                if not ranking_manager.should_crawl_url(url, CURRENT_CITY):
                    print(f"    🏆 랭킹 매니저: 중복 URL 스킵 (다른 탭에서 이미 크롤링됨)")
                    tab_skipped += 1
                    continue
            except Exception as e:
                print(f"    ⚠️ 랭킹 매니저 확인 실패: {e}")
            
            # 기존 중복 체크 (hashlib 고속 검사)
            if is_url_already_processed(url, CURRENT_CITY):
                print(f"    ⏭️ 이미 처리됨 - 건너뛰기")
                tab_skipped += 1
                continue
            
            # 상세 크롤링 실행 (32개 컬럼 + 듀얼 이미지)
            try:
                result = crawler.process_single_url(url, CURRENT_CITY, current_progress)
                
                if result.get('success') and not result.get('skipped'):
                    product_data = result.get('product_data', {})
                    product_name = product_data.get('상품명', 'N/A')[:30]
                    print(f"    ✅ 성공: {product_name}...")
                    
                    # 탭 및 랭킹 정보 추가
                    product_data['탭명'] = tab_name
                    product_data['탭내_랭킹'] = rank
                    
                    # 32개 컬럼 구조 확인
                    column_count = len(product_data.keys())
                    print(f"    📊 데이터 컬럼: {column_count}개")
                    
                    # 듀얼 이미지 확인
                    main_img = product_data.get('메인이미지_파일명', '정보 없음')
                    thumb_img = product_data.get('썸네일이미지_파일명', '정보 없음')
                    if main_img != '정보 없음' and thumb_img != '정보 없음':
                        print(f"    📸 듀얼 이미지: 메인 + 썸네일")
                    elif main_img != '정보 없음':
                        print(f"    📸 단일 이미지: 메인만")
                    else:
                        print(f"    📸 이미지: 없음")
                    
                    # 랭킹 매니저에 크롤링 완료 표시
                    try:
                        ranking_manager.mark_url_crawled(url, CURRENT_CITY)
                        print(f"    🏆 랭킹 매니저: 크롤링 완료 표시")
                    except Exception as e:
                        print(f"    ⚠️ 랭킹 완료 표시 실패: {e}")
                    
                    # 세션 상태 저장 (설정에 따라) - 새로 추가됨
                    if settings.get('save_session', True):
                        try:
                            from klook_modules.system_utils import save_crawler_state
                            session_data = {
                                'city': CURRENT_CITY,
                                'start_rank': START_RANK,
                                'end_rank': END_RANK,
                                'current_tab': tab_name,
                                'current_rank': rank,
                                'total_completed': total_completed + tab_completed + 1,
                                'settings': settings,
                                'timestamp': datetime.now().isoformat()
                            }
                            save_crawler_state(session_data, url)
                            print(f"    💾 세션 상태 저장 완료")
                        except Exception as e:
                            print(f"    ⚠️ 세션 저장 실패: {e}")
                    
                    tab_completed += 1
                    
                elif result.get('skipped'):
                    print(f"    ⏭️ 건너뛰기: {result.get('reason', 'unknown')}")
                    tab_skipped += 1
                    
                else:
                    print(f"    ❌ 실패: {result.get('error', '알 수 없음')}")
                    tab_failed += 1
                    
            except Exception as e:
                print(f"    💥 예외 발생: {e}")
                tab_failed += 1
            
            # 진행 상황 표시
            elapsed = time.time() - start_time
            avg_time = elapsed / max(total_completed + tab_completed, 1)
            remaining = (total_planned - current_progress) * avg_time
            
            print(f"    ⏱️ 경과: {int(elapsed//60)}분 | 예상 남은 시간: {int(remaining//60)}분")
            
            # 자연스러운 대기
            time.sleep(2)
        
        # 탭 완료 정리
        total_completed += tab_completed
        total_skipped += tab_skipped
        total_failed += tab_failed
        
        print(f"\n✅ '{tab_name}' 탭 완료!")
        print(f"   성공: {tab_completed}개 | 건너뜀: {tab_skipped}개 | 실패: {tab_failed}개")

except KeyboardInterrupt:
    print("\n⏹️ 사용자에 의해 중단되었습니다.")
    # 중단 시에도 세션 저장
    if settings.get('save_session', True):
        try:
            from klook_modules.system_utils import save_crawler_state
            interrupt_session_data = {
                'city': CURRENT_CITY,
                'start_rank': START_RANK,
                'end_rank': END_RANK,
                'total_completed': total_completed,
                'settings': settings,
                'interrupted': True,
                'timestamp': datetime.now().isoformat()
            }
            save_crawler_state(interrupt_session_data, "INTERRUPTED")
            print("💾 중단 상태 저장 완료 - Resume 기능으로 이어서 계속할 수 있습니다.")
        except Exception as e:
            print(f"⚠️ 중단 상태 저장 실패: {e}")
            
    # 중단 시에도 최종 백업 실행
    try:
        crawler.final_backup(CURRENT_CITY)
    except Exception as e:
        print(f"⚠️ 중단 시 백업 실패: {e}")
        
except Exception as e:
    print(f"\n💥 크롤링 중 오류: {e}")
    import traceback
    traceback.print_exc()

# 최종 백업 실행
try:
    print(f"\n💾 최종 백업 실행 중...")
    crawler.final_backup(CURRENT_CITY)
except Exception as e:
    print(f"⚠️ 최종 백업 실패: {e}")

# 최종 결과
end_time = time.time()
total_time = end_time - start_time

print("\n🎉 크롤링 완료!")
print("=" * 70)
print(f"📊 최종 결과:")
print(f"   ✅ 성공: {total_completed}개")
print(f"   ⏭️ 건너뜀: {total_skipped}개")
print(f"   ❌ 실패: {total_failed}개")
print(f"   📊 총계: {total_completed + total_skipped + total_failed}개")
print(f"   ⏱️ 소요 시간: {int(total_time//60)}분 {int(total_time%60)}초")

# 성공률 계산
if total_completed + total_failed > 0:
    success_rate = (total_completed / (total_completed + total_failed)) * 100
    print(f"   📈 성공률: {success_rate:.1f}%")

# 32개 컬럼 구조 및 듀얼 이미지 확인
print(f"\n💾 32개 컬럼 구조: {'✅ 적용됨' if total_completed > 0 else '⚠️ 확인 필요'}")
print(f"📸 듀얼 이미지 시스템: {'✅ 적용됨' if settings.get('download_images', True) else '❌ 비활성화'}")

# 랭킹 매니저 최종 통계
try:
    ranking_stats = ranking_manager.get_city_ranking_stats(CURRENT_CITY)
    if ranking_stats:
        print(f"🏆 랭킹 매니저 최종 통계:")
        print(f"   총 URL: {ranking_stats.get('total_urls', 0)}개")
        print(f"   중복 URL: {ranking_stats.get('duplicate_urls', 0)}개")
        print(f"   처리된 탭: {len(ranking_stats.get('tabs_processed', []))}개")
except Exception as e:
    print(f"⚠️ 랭킹 통계 조회 실패: {e}")

print(f"\n📁 데이터 저장 위치:")
print(f"   - CSV: data/{CURRENT_CITY}/")
print(f"   - 이미지: klook_thumb_img/")
print(f"   - 랭킹 데이터: ranking_data/")
print(f"   - 세션 상태: crawler_state/")
print(f"   - 백업 파일: data/{CURRENT_CITY}/*_backup_*.csv")

## 🗺️ 6. Sitemap URL 추가 수집 (선택사항)

In [ ]:
# Sitemap에서 추가 URL 수집
print("🗺️ Sitemap에서 추가 URL 수집 옵션")
print("=" * 50)

sitemap_choice = input("Sitemap에서 추가 URL을 수집하시겠습니까? (y/n): ")

if sitemap_choice.lower() == 'y':
    from klook_modules.url_collection import collect_urls_from_sitemap
    
    print(f"\n🗺️ '{CURRENT_CITY}' Sitemap URL 수집 중...")
    
    # Sitemap URL 수집
    sitemap_urls = collect_urls_from_sitemap(CURRENT_CITY, limit=500)
    
    if sitemap_urls:
        print(f"📊 Sitemap에서 {len(sitemap_urls)}개 URL 발견")
        
        # 중복 제거 (이미 크롤링한 URL 제외)
        new_urls = []
        for url in sitemap_urls:
            if not is_url_already_processed(url, CURRENT_CITY):
                new_urls.append(url)
        
        print(f"🆕 새로운 URL: {len(new_urls)}개")
        
        if new_urls:
            proceed = input(f"\n{len(new_urls)}개의 새로운 URL을 크롤링하시겠습니까? (y/n): ")
            
            if proceed.lower() == 'y':
                print("\n🚀 Sitemap URL 크롤링 시작...")
                
                sitemap_completed = 0
                sitemap_failed = 0
                
                for i, url in enumerate(new_urls, 1):
                    print(f"\n📊 진행률: {i}/{len(new_urls)} | Sitemap URL")
                    print(f"🔗 URL: {url[:60]}...")
                    
                    try:
                        # 수정된 함수 호출: process_single_url 사용
                        result = crawler.process_single_url(url, CURRENT_CITY, f"sitemap_{i}")
                        
                        if result.get('success') and not result.get('skipped'):
                            product_data = result.get('product_data', {})
                            product_name = product_data.get('상품명', 'N/A')[:30]
                            print(f"    ✅ 성공: {product_name}...")
                            
                            # 탭 정보 업데이트
                            # product_data는 이미 CSV에 저장됨 (process_single_url에서 자동 처리)
                            
                            sitemap_completed += 1
                            
                        elif result.get('skipped'):
                            print(f"    ⏭️ 건너뛰기: {result.get('reason', 'unknown')}")
                        else:
                            print(f"    ❌ 실패: {result.get('error', '알 수 없음')}")
                            sitemap_failed += 1
                            
                    except Exception as e:
                        print(f"    💥 예외: {e}")
                        sitemap_failed += 1
                    
                    time.sleep(1)
                
                print(f"\n✅ Sitemap 크롤링 완료!")
                print(f"   성공: {sitemap_completed}개 | 실패: {sitemap_failed}개")
            else:
                print("⏭️ Sitemap 크롤링을 건너뜁니다.")
        else:
            print("ℹ️ 새로운 URL이 없습니다.")
    else:
        print("❌ Sitemap에서 URL을 찾을 수 없습니다.")
else:
    print("⏭️ Sitemap 수집을 건너뜁니다.")

## 📊 7. 크롤링 결과 분석 및 보고서

In [ ]:
# 크롤링 결과 분석 및 보고서 생성
print("📊 크롤링 결과 분석 중...")
print("=" * 50)

try:
    from klook_modules.system_utils import get_hash_stats
    from klook_modules.url_manager import get_url_collection_stats
    import pandas as pd
    
    # 1. 해시 시스템 통계
    hash_stats = get_hash_stats(CURRENT_CITY)
    print(f"🔒 해시 시스템 통계:")
    print(f"   처리된 URL: {hash_stats.get('processed_count', 0)}개")
    
    # 2. URL 수집 통계
    url_stats = get_url_collection_stats(CURRENT_CITY)
    print(f"\n🔗 URL 수집 통계:")
    print(f"   수집 파일: {url_stats.get('total_files', 0)}개")
    print(f"   총 URL: {url_stats.get('total_urls', 0)}개")
    print(f"   최근 수집: {url_stats.get('latest_collection', 'N/A')}")
    
    # 3. CSV 데이터 분석 (새로운 32개 컬럼 구조)
    from klook_modules.config import get_city_info
    continent, country = get_city_info(CURRENT_CITY)
    
    # 새로운 파일명 형식 확인
    if CURRENT_CITY in ["마카오", "홍콩", "싱가포르"]:
        csv_path_new = f"data/{continent}/{CURRENT_CITY}_klook_products_all.csv"
        csv_path_old = f"data/{continent}/klook_{CURRENT_CITY}_products.csv"
    else:
        csv_path_new = f"data/{continent}/{country}/{CURRENT_CITY}/{CURRENT_CITY}_klook_products_all.csv"
        csv_path_old = f"data/{continent}/{country}/{CURRENT_CITY}/klook_{CURRENT_CITY}_products.csv"
    
    # 새 구조 CSV 확인
    csv_path = csv_path_new if os.path.exists(csv_path_new) else csv_path_old
    csv_structure = "32개 컬럼 (신규)" if os.path.exists(csv_path_new) else "13개 컬럼 (기존)"
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, encoding='utf-8-sig')
        
        print(f"\n📋 CSV 데이터 분석:")
        print(f"   총 상품: {len(df)}개")
        print(f"   구조: {csv_structure}")
        print(f"   컬럼 수: {len(df.columns)}개")
        print(f"   파일 위치: {csv_path}")
        
        # 32개 컬럼 구조 상세 분석
        if len(df.columns) >= 30:  # 32개 컬럼 구조
            print(f"\n💾 32개 컬럼 구조 확인:")
            
            # 기본 정보 컬럼
            basic_cols = ['번호', '도시ID', '상품명', '가격_원본', '가격_정제']
            basic_present = [col for col in basic_cols if col in df.columns]
            print(f"   기본 정보: {len(basic_present)}/{len(basic_cols)}개")
            
            # 이미지 컬럼 (8개)
            image_cols = [col for col in df.columns if '이미지' in col]
            print(f"   이미지 정보: {len(image_cols)}개")
            
            # 랭킹 컬럼
            ranking_cols = ['탭명', '탭순서', '탭내_랭킹', 'URL_해시']
            ranking_present = [col for col in ranking_cols if col in df.columns]
            print(f"   랭킹 정보: {len(ranking_present)}/{len(ranking_cols)}개")
            
            # 듀얼 이미지 통계
            if '메인이미지_파일명' in df.columns and '썸네일이미지_파일명' in df.columns:
                main_images = df[df['메인이미지_파일명'] != '정보 없음']
                thumb_images = df[df['썸네일이미지_파일명'] != '정보 없음']
                dual_images = df[(df['메인이미지_파일명'] != '정보 없음') & 
                               (df['썸네일이미지_파일명'] != '정보 없음')]
                
                print(f"\n📸 듀얼 이미지 통계:")
                print(f"   메인 이미지: {len(main_images)}개")
                print(f"   썸네일 이미지: {len(thumb_images)}개")
                print(f"   듀얼 이미지: {len(dual_images)}개")
        
        # 탭별 분석
        tab_col = '탭명' if '탭명' in df.columns else None
        if tab_col:
            tab_counts = df[tab_col].value_counts()
            print(f"\n🎪 탭별 상품 수:")
            for tab, count in tab_counts.items():
                print(f"   {tab}: {count}개")
        
        # 가격 분석 (구 컬럼/신 컬럼 호환)
        price_col = '가격_정제' if '가격_정제' in df.columns else '가격'
        if price_col in df.columns:
            valid_prices = df[df[price_col] != '정보 없음'][price_col]
            print(f"\n💰 가격 정보:")
            print(f"   가격 있음: {len(valid_prices)}개")
            print(f"   가격 없음: {len(df) - len(valid_prices)}개")
        
        # 평점 분석 (구 컬럼/신 컬럼 호환)
        rating_col = '평점_정제' if '평점_정제' in df.columns else '평점'
        if rating_col in df.columns:
            valid_ratings = df[df[rating_col] != '정보 없음'][rating_col]
            print(f"\n⭐ 평점 정보:")
            print(f"   평점 있음: {len(valid_ratings)}개")
            print(f"   평점 없음: {len(df) - len(valid_ratings)}개")
        
        # 추가 정보 분석 (32개 컬럼 구조)
        if '하이라이트' in df.columns:
            highlights = df[df['하이라이트'] != '정보 없음']
            print(f"   하이라이트: {len(highlights)}개")
        
        if '언어' in df.columns:
            languages = df[df['언어'] != '정보 없음']
            print(f"   언어 정보: {len(languages)}개")
        
        # 최근 크롤링 시간
        time_col = '수집_시간' if '수집_시간' in df.columns else '수집일시'
        if time_col in df.columns:
            latest_crawl = df[time_col].max()
            print(f"\n⏰ 최근 크롤링: {latest_crawl}")
    
    else:
        print(f"\n⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
    
    # 4. 랭킹 매니저 분석
    try:
        ranking_stats = ranking_manager.get_city_ranking_stats(CURRENT_CITY)
        if ranking_stats:
            print(f"\n🏆 랭킹 매니저 분석:")
            print(f"   총 관리 URL: {ranking_stats.get('total_urls', 0)}개")
            print(f"   중복 발견 URL: {ranking_stats.get('duplicate_urls', 0)}개")
            print(f"   처리된 탭: {ranking_stats.get('tabs_processed', [])}")
            print(f"   마지막 업데이트: {ranking_stats.get('last_updated', 'N/A')}")
    except Exception as e:
        print(f"⚠️ 랭킹 분석 실패: {e}")
    
    # 5. 이미지 폴더 분석
    try:
        img_base_folder = "klook_thumb_img"
        if os.path.exists(img_base_folder):
            if CURRENT_CITY in ["마카오", "홍콩", "싱가포르"]:
                img_folder = os.path.join(img_base_folder, continent)
            else:
                img_folder = os.path.join(img_base_folder, continent, country, CURRENT_CITY)
            
            if os.path.exists(img_folder):
                img_files = [f for f in os.listdir(img_folder) if f.endswith('.jpg')]
                main_imgs = [f for f in img_files if '_thumb' not in f and '_main' not in f]
                thumb_imgs = [f for f in img_files if '_thumb' in f]
                
                print(f"\n📸 이미지 폴더 분석:")
                print(f"   폴더 위치: {img_folder}")
                print(f"   총 이미지: {len(img_files)}개")
                print(f"   메인 이미지: {len(main_imgs)}개")
                print(f"   썸네일: {len(thumb_imgs)}개")
            else:
                print(f"\n📸 이미지 폴더 없음: {img_folder}")
    except Exception as e:
        print(f"⚠️ 이미지 분석 실패: {e}")

except Exception as e:
    print(f"❌ 분석 중 오류: {e}")

# 세션 보고서 생성 (업데이트된 정보 포함)
try:
    session_report = {
        "city": CURRENT_CITY,
        "mode": CRAWLING_MODE,
        "start_rank": START_RANK,
        "end_rank": END_RANK,
        "completed_at": datetime.now().isoformat(),
        "settings": settings,
        "total_time_minutes": int(total_time // 60),
        "results": {
            "completed": total_completed,
            "skipped": total_skipped,
            "failed": total_failed
        },
        "system_info": {
            "column_structure": "32개 컬럼",
            "dual_image_system": settings.get('download_images', True),
            "ranking_manager": True,
            "hashlib_system": CONFIG.get('USE_HASH_SYSTEM', True)
        }
    }
    
    # 보고서 저장
    os.makedirs("session_reports", exist_ok=True)
    report_filename = f"session_reports/klook_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    import json
    with open(report_filename, 'w', encoding='utf-8') as f:
        json.dump(session_report, f, ensure_ascii=False, indent=2)
    
    print(f"\n📋 세션 보고서 저장: {report_filename}")

except Exception as e:
    print(f"⚠️ 보고서 저장 실패: {e}")

print("\n🎉 모든 작업이 완료되었습니다!")
print(f"💾 32개 컬럼 구조: {'✅ 적용됨' if csv_structure == '32개 컬럼 (신규)' else '⚠️ 기존 구조'}")
print(f"📸 듀얼 이미지: {'✅ 적용됨' if settings.get('download_images', True) else '❌ 비활성화'}")
print(f"🏆 랭킹 매니저: {'✅ 활성화' if 'ranking_manager' in locals() else '❌ 비활성화'}")

## 🧹 8. 시스템 정리 및 종료

In [ ]:
# 시스템 정리 및 브라우저 종료
print("🧹 시스템 정리 중...")

try:
    # 크롤러 정리
    if 'crawler' in locals():
        crawler.cleanup()
        print("✅ 크롤러 정리 완료")
    
    # 드라이버 종료
    if 'driver' in locals():
        driver.quit()
        print("✅ 브라우저 종료 완료")
    
    # 마스터 컨트롤러 정리
    if 'controller' in locals():
        controller.cleanup_system()
        print("✅ 시스템 정리 완료")

except Exception as e:
    print(f"⚠️ 정리 중 오류: {e}")

print("\n👋 KLOOK 크롤러를 종료합니다.")
print("\n" + "="*70)
print("🎉 크롤링 세션이 성공적으로 완료되었습니다!")
print("📁 결과 파일들을 확인하세요:")
print(f"   - CSV 데이터: data/{CURRENT_CITY}/")
print(f"   - 세션 보고서: session_reports/")
print(f"   - URL 수집 로그: url_collected/")
print("="*70)

---

## 🔄 이어서 계속하기 (Resume 기능)

만약 크롤링이 중간에 중단되었다면, 다음 셀을 실행하여 이어서 계속할 수 있습니다:

In [ ]:
# Resume 기능 - 중단된 지점부터 이어서 계속
print("🔄 이어서 계속하기 기능")
print("=" * 40)

resume_choice = input("이전 세션을 이어서 계속하시겠습니까? (y/n): ")

if resume_choice.lower() == 'y':
    try:
        from klook_modules.system_utils import load_session_state
        
        # 사용자에게 도시명 입력 받기
        resume_city = input("이어서 계속할 도시명을 입력하세요: ")
        
        # 이전 세션 상태 로드
        session_state = load_session_state(resume_city)
        
        if session_state:
            print(f"✅ '{resume_city}' 이전 세션 발견!")
            print(f"📊 이전 세션 정보: {session_state.get('timestamp', 'N/A')}")
            
            # 설정 복원 (수정된 변수명)
            CURRENT_CITY = resume_city
            START_RANK = session_state.get('start_rank', 1)
            END_RANK = session_state.get('end_rank', 50)
            CRAWLING_MODE = 'resume'
            
            # 설정 딕셔너리 복원
            settings = {
                'city': resume_city,
                'start_rank': START_RANK,
                'end_rank': END_RANK,
                'mode': 'resume',
                'skip_duplicates': True,
                'auto_save': session_state.get('auto_save', True),
                'download_images': session_state.get('download_images', True),
                'save_session': True
            }
            
            print(f"🔄 '{CURRENT_CITY}'에서 이어서 계속합니다...")
            print(f"📊 복원된 순위 범위: {START_RANK}위 ~ {END_RANK}위")
            print(f"💾 32개 컬럼 구조: ✅ 적용")
            print(f"🏆 랭킹 매니저: ✅ 활성화")
            print("위의 '3. 크롤링 실행' 셀부터 다시 실행하세요.")
        else:
            print(f"❌ '{resume_city}'의 이전 세션을 찾을 수 없습니다.")
            
    except Exception as e:
        print(f"❌ Resume 기능 오류: {e}")
else:
    print("ℹ️ 새로운 세션을 시작하세요.")